In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
np.random.seed(12345)

In [92]:
!pip install bs4

  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/1d/5d/3260694a59df0ec52f8b4883f5d23b130bc237602a1411fa670eae12351e/beautifulsoup4-4.7.1-py3-none-any.whl
  Running setup.py bdist_wheel for bs4 ... done
  Stored in directory: /home/daniel/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [103]:
from bs4 import BeautifulSoup

In [104]:
str(2).zfill(2)

'02'

In [237]:
data = []
for i in range(1, 16):
    with open('../reuters/reut2-0{}.sgm'.format(str(i).zfill(2), 'r')) as f:
        print(i)
        bs = BeautifulSoup(f, from_encoding='latin1')
        for ret in bs.find_all('reuters'):
            places = [p.text for p in ret.find('places').find_all('d')]
            for p in places:
                data.append({'is_uk': 'uk' in places, 'text': ret.find('text').text})
                
data = pd.DataFrame(data)
data.text = data.text.str.lower().str.replace("[^A-Za-z ]+", "").str.replace("\s+", " ")
data['is_uk'] = data.is_uk.astype(int)
uk_data = data[data.is_uk==1]
non_uk_data = data[data.is_uk == 0].head(uk_data.shape[0])
data = pd.concat([uk_data, non_uk_data]).reset_index(drop=True)
data = data.loc[np.random.choice(data.index, size=data.shape[0])].reset_index(drop=True)
data.head(500).to_csv("./uk.csv", index=False)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [185]:
data = []
for i in range(1, 16):
    with open('../reuters/reut2-0{}.sgm'.format(str(i).zfill(2), 'r')) as f:
        print(i)
        bs = BeautifulSoup(f, from_encoding='latin1')
        for ret in bs.find_all('reuters'):
            places = [p.text for p in ret.find('topics').find_all('d')]
            for p in places:
                data.append({'is_earn': 'earn' in places, 'text': ret.find('text').text})
                
data = pd.DataFrame(data)
data['is_earn'] = data.is_earn.astype(int)
data['text'] = data.text.str.replace("\n", "")
data['text'] = data.text.str.lower().str.replace("[^A-Za-z ]+", "").str.replace("\s+", " ")
uk_data = data[data.is_earn==1]
non_uk_data = data[data.is_earn == 0].head(uk_data.shape[0])
data = pd.concat([uk_data, non_uk_data]).reset_index(drop=True)
data = data.loc[np.random.choice(data.index, size=data.shape[0])].reset_index(drop=True)

data.head(500).to_csv("earnings.csv", index=False)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [256]:
data = pd.read_csv("./uk.csv").head(500)

In [257]:
from sklearn.feature_extraction.text import CountVectorizer

In [258]:
vect = CountVectorizer()

In [259]:
res = vect.fit_transform(data.text)

In [241]:
vect.transform(['hello', 'uk']).toarray()[:, vect.vocabulary_['uk']]

array([0, 1])

In [242]:
data.drop('heuristic', axis=1, inplace=True)

ValueError: labels ['heuristic'] not contained in axis

In [243]:
data.to_csv("./reuters_with_place.csv", index=False)
data.drop('place', axis=1).to_csv("./reuters_uk.csv", index=False)

ValueError: labels ['place'] not contained in axis

In [212]:
data = pd.read_csv("./reuters_uk.csv")

In [213]:
uk_data = data[data.is_uk==1]
non_uk_data = data[data.is_uk == 0].head(uk_data.shape[0])
data = pd.concat([uk_data, non_uk_data]).reset_index(drop=True)
data = data.loc[np.random.choice(data.index, size=data.shape[0])].reset_index(drop=True)


In [261]:
X = (data.text.str.contains('united kingdom') | data.text.str.contains('uk')).values[:, np.newaxis]
y = data['is_uk'].values
X = data.text.values

In [262]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
np.random.seed(12345)


In [263]:
def train_test_split(X, y):
    N = X.shape[0]
    split_size = int(N/5)
    split = int(N - 2*split_size)
    train_X = X[:split]
    train_y = y[:split]
    val_X = X[split:split+split_size]
    val_y = y[split:split+split_size]
    test_X = X[split+split_size:]
    test_y = y[split+split_size:]
    
    return train_X, train_y, val_X, val_y, test_X, test_y

In [264]:
train_X, train_y, val_X, val_y, test_X, test_y = train_test_split(X, y)

In [265]:
def test_model(model, params, train_X, train_y, test_X, test_y):
    scorer = make_scorer(f1_score, greater_is_better=True) # Use mean squared error as score
    gs = GridSearchCV(model, params, scoring=scorer, cv=StratifiedKFold(n_splits=5, shuffle=True)) # Cross-validation to pick best hyperparameter
    gs.fit(train_X, train_y)
    gs.best_estimator_.fit(train_X, train_y)
    train_pred = gs.best_estimator_.predict(train_X) # Make prediction on training set
    test_pred = gs.best_estimator_.predict(test_X) # Make prediction on test set
    print("Training score: ", f1_score(train_y, train_pred))
    print("Test score: ", f1_score(test_y, test_pred))
    print("Test accuracy: ", accuracy_score(test_y, test_pred))
    return gs.best_estimator_

In [266]:
model = Pipeline([('features', None),
                  ('preprocess', None),
                  ('model', None)])


In [267]:
params = dict(model=[SVC(class_weight='balanced')], model__C=np.logspace(-1, 1))
test_model(model, params, train_X, train_y, val_X, val_y)

ValueError: could not convert string to float: 'moscow carries out nuclear test london march the soviet union carried out a nucleartest early today the official tass news agency reported according to the report monitored by the britishbroadcasting corporation the explosion was at gmt a blast on february ended a month unilateral testmoratorium declared by the soviet union moscow blamed the endof the freeze on us refusal to join a total test ban tass said the latest explosion with a power of up to kilotonnes had the aim of improving military equipment reuter'

In [268]:
vect = CountVectorizer()

In [269]:
vect.fit_transform(train_X).mean(axis=1).shape

(300, 1)

In [270]:
from sklearn.ensemble import RandomForestClassifier

In [271]:
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer

In [272]:
params = dict(model=[LinearSVC(class_weight='balanced')],features=[CountVectorizer(stop_words='english'), CountVectorizer(), HashingVectorizer(), TfidfVectorizer()])
best_model = test_model(model, params, train_X, train_y, val_X, val_y)

Training score:  1.0
Test score:  0.9032258064516128
Test accuracy:  0.91


In [274]:
params = dict(model=[RandomForestClassifier(class_weight='balanced')],features=[CountVectorizer(), HashingVectorizer(), TfidfVectorizer()])
best_model = test_model(model, params, train_X, train_y, val_X, val_y)

Training score:  1.0
Test score:  0.8791208791208791
Test accuracy:  0.89


In [141]:
best_model.fit(np.hstack((train_X, val_X)), np.hstack((train_y, val_y)))

Pipeline(memory=None,
     steps=[('features', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        s...imators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])

In [142]:
f1_score(test_y, best_model.predict(test_X))

0.9839704069050554

In [143]:
imps = best_model.named_steps['model'].feature_importances_

In [148]:
imp_order=  np.argsort(imps)[::-1]

In [149]:
f_names = np.asarray(list(best_model.named_steps['features'].vocabulary_.keys()))